In [1]:
from nbed.driver import NbedDriver
from pathlib import Path

args = {
    "geometry": str(Path("molecular_structures/cyclopentane.xyz")),
    "basis": "STO-3G",
    "xc_functional": "b3lyp",
    "n_active_atoms":2,
    "projector": "mu",
    "localization": "spade",
    "convergence": 1e-6,
    "savefile": None,
    "run_ccsd_emb": True,
    "run_fci_emb": True,
    'charge':0,
    'spin':0,
}

driver = NbedDriver(
    **args,
)

/home/mwilliams/anaconda3/envs/nbed/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'
/home/mwilliams/anaconda3/envs/nbed/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
import logging
logger = logging.getLogger(__name__)
from nbed.localizers import Localizer
from typing import Optional, Tuple, Dict
import numpy as np
from pyscf import scf, dft, gto
from scipy import linalg

class OldSPADELocalizer(Localizer):
    """Localizer Class to carry out SPADE"""

    def __init__(
        self,
        pyscf_scf: gto.Mole,
        n_active_atoms: int,
        occ_cutoff: Optional[float] = 0.95,
        virt_cutoff: Optional[float] = 0.95,
        run_virtual_localization: Optional[bool] = False,
    ):
        super().__init__(
            pyscf_scf,
            n_active_atoms,
            occ_cutoff=occ_cutoff,
            virt_cutoff=virt_cutoff,
            run_virtual_localization=run_virtual_localization,
        )

    def _localize(
        self,
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """Localise orbitals using SPADE.
        Returns:
            active_MO_inds (np.array): 1D array of active occupied MO indices
            enviro_MO_inds (np.array): 1D array of environment occupied MO indices
            c_active (np.array): C matrix of localized occupied active MOs (columns define MOs)
            c_enviro (np.array): C matrix of localized occupied ennironment MOs
            c_loc_occ (np.array): full C matrix of localized occupied MOs
        """
        logger.info("Localising with SPADE.")
        n_occupied_orbitals = np.count_nonzero(self._global_ks.mo_occ == 2)
        occupied_orbitals = self._global_ks.mo_coeff[:, :n_occupied_orbitals]

        n_act_aos = self._global_ks.mol.aoslice_by_atom()[self._n_active_atoms - 1][-1]
        logger.debug(f"{n_act_aos} active AOs.")

        ao_overlap = self._global_ks.get_ovlp()

        # Orbital rotation and partition into subsystems A and B
        # rotation_matrix, sigma = embed.orbital_rotation(occupied_orbitals,
        #    n_act_aos, ao_overlap)

        rotated_orbitals = (
            linalg.fractional_matrix_power(ao_overlap, 0.5) @ occupied_orbitals
        )
        _, sigma, right_vectors = linalg.svd(rotated_orbitals[:n_act_aos, :])

        logger.debug(f"Singular Values: {sigma}")

        # n_act_mos, n_env_mos = embed.orbital_partition(sigma)
        # Prevents an error with argmax
        if len(sigma) == 1:
            n_act_mos = 1
        else:
            value_diffs = sigma[:-1] - sigma[1:]
            n_act_mos = np.argmax(value_diffs) + 1
        n_env_mos = n_occupied_orbitals - n_act_mos
        logger.debug(f"{n_act_mos} active MOs.")
        logger.debug(f"{n_env_mos} environment MOs.")

        # get active and enviro indices
        active_MO_inds = np.arange(n_act_mos)
        enviro_MO_inds = np.arange(n_act_mos, n_act_mos + n_env_mos)

        # Defining active and environment orbitals and density
        c_active = occupied_orbitals @ right_vectors.T[:, :n_act_mos]
        c_enviro = occupied_orbitals @ right_vectors.T[:, n_act_mos:]
        c_loc_occ = occupied_orbitals @ right_vectors.T

        # storing condition used to select env system
        self.enviro_selection_condition = sigma

        alpha = active_MO_inds, enviro_MO_inds, c_active, c_enviro, c_loc_occ
        beta = None

        return (alpha, beta)

In [3]:
old = OldSPADELocalizer(driver._global_ks, driver.n_active_atoms)

KeyboardInterrupt: 

In [ ]:
old.active_MO_inds

In [2]:
import logging
logger = logging.getLogger(__name__)
from nbed.localizers import Localizer
from typing import Optional, Tuple, Dict
import numpy as np
from pyscf import scf, dft, gto

init_huzinaga_rhf_with_mu = False

"""Generate embedded Hamiltonian.

Note run_mu_shift (bool) and run_huzinaga (bool) flags define which method to use (can be both)
This is done when object is initialized.
"""
logger.debug("Embedding molecule.")
localized_system = driver._localize()
driver.localized_system = localized_system

# logger.info(localized_system.active_MO_inds, localized_system.beta_active_MO_inds)
# logger.info(localized_system.enviro_MO_inds, localized_system.beta_enviro_MO_inds)

e_nuc = driver._global_ks.energy_nuc()

# Run subsystem DFT (calls localized rks)
#driver._subsystem_dft()


occupancy=array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])
occupancy=array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])


In [ ]:
print(np.shape(localized_system.c_active))
print(localized_system.active_MO_inds)
print(localized_system.beta_c_active)

driver.e_act=-129.12067009902808 e_xc_act=-6.870585748333742 

j_act=array([[ 1.22472929e+01,  2.41156517e+00,  8.78839625e-04, ...,
         3.84411610e-03,  3.31862315e-03,  1.51435460e-03],
       [ 2.41156517e+00,  5.95052145e+00,  8.14159732e-03, ...,
         3.53253934e-02,  5.43706112e-02,  3.19017803e-02],
       [ 8.78839625e-04,  8.14159732e-03,  5.94637095e+00, ...,
         5.10149782e-02,  7.48919853e-02,  4.38669859e-02],
       ...,
       [ 3.84411610e-03,  3.53253934e-02,  5.10149782e-02, ...,
         2.13277578e+00,  7.35762625e-01, -6.59146759e-01],
       [ 3.31862315e-03,  5.43706112e-02,  7.48919853e-02, ...,
         7.35762625e-01,  1.72918274e+00,  3.00287207e-01],
       [ 1.51435460e-03,  3.19017803e-02,  4.38669859e-02, ...,
        -6.59146759e-01,  3.00287207e-01,  1.62770956e+00]])

driver.e_env=-329.78536772993164 e_xc_env=-23.580761027571594 

j_env=array([[ 7.53028368e+00,  1.86998591e+00,  7.78097492e-02, ...,
         3.07296498e-03,  2.69757415e-03,  1.24786844e-03],
       [ 1.86998591e+00,  7.50747791e+00,  8.94502957e-01, ...,
         7.15294590e-02,  1.43805368e-01,  9.38820475e-02],
       [ 7.78097492e-02,  8.94502957e-01,  7.59612955e+00, ...,
         1.18488504e-01,  2.22489217e-01,  1.43698427e-01],
       ...,
       [ 3.07296498e-03,  7.15294590e-02,  1.18488504e-01, ...,
         1.10597928e+01,  3.58088013e+00, -3.46368435e+00],
       [ 2.69757415e-03,  1.43805368e-01,  2.22489217e-01, ...,
         3.58088013e+00,  8.37845713e+00,  1.57014876e+00],
       [ 1.24786844e-03,  9.38820475e-02,  1.43698427e-01, ...,
        -3.46368435e+00,  1.57014876e+00,  8.44687034e+00]])

e_xc_total=-30.801124900358385, e_xc_act=-6.870585748333742, e_xc_env=-23.580761027571594
j_cross=76.06825318343016 k_cross=0.0 xc_cross=-0.34977812445304934
-194.079748544697 -194.07974854469694


In [6]:
"""Function to perform subsystem RKS DFT calculation."""
logger.debug("Calculating active and environment subsystem terms.")

def _ks_components(
    ks_system: Localizer,
    subsystem_dm: np.ndarray,
) -> Tuple[float, float, np.ndarray, np.ndarray, np.ndarray]:
    """Calculate the components of subsystem energy from a RKS DFT calculation.

    For a given density matrix this function returns the electronic energy, exchange correlation energy and
    J,K, V_xc matrices.

    Args:
        dm_matrix (np.ndarray): density matrix (to calculate all matrices from)

    Returns:
        Energy_elec (float): DFT energy defubed by input density matrix
        e_xc (float): exchange correlation energy defined by input density matrix
        J_mat (np.ndarray): J_matrix defined by input density matrix
    """
    logger.debug("Finding subsystem RKS componenets.")
    # It seems that PySCF lumps J and K in the J array
    # need to access the potential for the right subsystem for unrestricted
    two_e_term = ks_system.get_veff(dm=subsystem_dm)
    j_mat = two_e_term.vj
    # k_mat = np.zeros_like(j_mat)

    e_xc = two_e_term.exc
    # v_xc = two_e_term - j_mat

    energy_elec = (
        np.einsum("ij,ji->", ks_system.get_hcore(), subsystem_dm)
        # + two_e_term.ecoul
        # + two_e_term.exc
    )

    # if check_E_with_pyscf:
    #     energy_elec_pyscf = driver._global_ks.energy_elec(dm=dm_matrix)[0]
    #     if not np.isclose(energy_elec_pyscf, energy_elec):
    #         raise ValueError("Energy calculation incorrect")
    logger.debug(f"Subsystem RKS components found.")
    return energy_elec, e_xc, j_mat, two_e_term.ecoul

(alpha_e_act, alpha_e_xc_act, alpha_j_act, alpha_ecoul_act) = _ks_components(
    driver._global_ks, driver.localized_system.dm_active
)
print(alpha_e_act, alpha_e_xc_act, alpha_ecoul_act)
(alpha_e_env, alpha_e_xc_env, alpha_j_env, alpha_ecoul_env) = _ks_components(
    driver._global_ks, driver.localized_system.dm_enviro
)
print(alpha_e_env, alpha_e_xc_env, alpha_ecoul_env)
driver.e_act = alpha_e_act
driver.e_env = alpha_e_env

if not driver._restricted_scf:
    (beta_e_act, beta_e_xc_act, beta_j_act, beta_ecoul_act) = _ks_components(
        driver._global_ks, driver.localized_system.beta_dm_active
    )
    print(beta_e_act, beta_e_xc_act, beta_ecoul_act)
    (beta_e_env, beta_e_xc_env, beta_j_env, beta_ecoul_env) = _ks_components(
        driver._global_ks, driver.localized_system.beta_dm_enviro
    )
    print(beta_e_env, beta_e_xc_env, beta_ecoul_env)
    driver.e_act += beta_e_act
    driver.e_env += beta_e_env

# Computing cross subsystem terms
logger.debug("Calculating two electron cross subsystem energy.")
total_dm = driver.localized_system.dm_active + driver.localized_system.dm_enviro
if not driver._restricted_scf:
    total_dm += driver.localized_system.beta_dm_active + driver.localized_system.beta_dm_enviro
    
two_e_term_total = driver._global_ks.get_veff(
    dm=total_dm)
e_xc_total = two_e_term_total.exc

# if not driver._restricted_scf:
#     beta_two_e_term_total = driver._global_ks.get_veff(
#         dm=driver.localized_system.beta_dm_active + driver.localized_system.beta_dm_enviro
#     )
#     e_xc_total = (e_xc_total + beta_two_e_term_total.exc)/2

j_cross = 0.5 * (
    np.einsum("ij,ij", driver.localized_system.dm_active, alpha_j_env)
    + np.einsum("ij,ij", driver.localized_system.dm_enviro, alpha_j_act)
)
if not driver._restricted_scf:
    beta_j_cross = 0.5 * (
        np.einsum("ij,ij", driver.localized_system.dm_active, beta_j_env)
        + np.einsum("ij,ij", driver.localized_system.dm_enviro, beta_j_act)
        + np.einsum("ij,ij", driver.localized_system.beta_dm_active, beta_j_env)
        + np.einsum("ij,ij", driver.localized_system.beta_dm_enviro, beta_j_act)
        + np.einsum("ij,ij", driver.localized_system.beta_dm_active, alpha_j_env)  
        + np.einsum("ij,ij", driver.localized_system.beta_dm_enviro, alpha_j_act)
    )
    j_cross += beta_j_cross

# Because of projection
k_cross = 0.0

xc_cross = e_xc_total - alpha_e_xc_act - alpha_e_xc_env #- beta_e_xc_act - beta_e_xc_env
# if not driver._restricted_scf:
    # xc_cross = beta_e_xc_act - beta_e_xc_env
# driver.e_act=-129.12067009902808      driver.e_env=-329.78536772993164
# e_xc_total=-30.801124900358385, e_xc_act=-6.870585748333742, e_xc_env=-23.580761027571594
# j_cross=76.06825318343016 k_cross=0.0 xc_cross=-0.34977812445304934

# driver.e_act=-143.82987154686245 driver.e_env=-411.35820634557757
# e_xc_total=-30.80112219655959, alpha_e_xc_act=-2.6149703376981797, alpha_e_xc_env=-8.98550003899027, 
# j_cross=76.06820092620214, k_cross=0.0, xc_cross=-19.20065181987114


if not driver._restricted_scf:
    print(beta_e_xc_act, beta_e_xc_env)

# overall two_electron cross energy
driver.two_e_cross = j_cross + k_cross + xc_cross
print(f"{driver.e_act=} {driver.e_env=}")
print(f"{e_xc_total=}, {alpha_e_xc_act=}, {alpha_e_xc_env=}, {j_cross=}, {k_cross=}, {xc_cross=}")

energy_DFT_components = (
    driver.e_act + driver.e_env + driver.two_e_cross + driver._global_ks.energy_nuc()
)
logger.info("RKS components")
logger.info(driver.e_act)
logger.info(driver.e_env)
logger.info(driver.two_e_cross)
logger.info(driver._global_ks.energy_nuc())
print(energy_DFT_components)
print(driver._global_ks.e_tot)
if not np.isclose(energy_DFT_components, driver._global_ks.e_tot):
    logger.error(
        "DFT energy of localized components not matching supersystem DFT."
    )
    raise ValueError(
        "DFT energy of localized components not matching supersystem DFT."
    )


-77.47494636976423 -2.6149703376981797 8.174934549887327
-240.28519988302781 -8.98550003899027 43.59144604618194
-77.4748190375258 -2.61496670185692 8.174896350095347
-240.28470023949086 -8.985487296065548 43.59123506581496


1970-01-01 02:57:30,092: __main__: ERROR: DFT energy of localized components not matching supersystem DFT.


-2.61496670185692 -8.985487296065548
driver.e_act=-154.94976540729004 driver.e_env=-480.5699001225187
e_xc_total=-30.80112219655959, alpha_e_xc_act=-2.6149703376981797, alpha_e_xc_env=-8.98550003899027, j_cross=76.06820092620214, k_cross=0.0, xc_cross=-19.20065181987114
-389.54430219819204
-194.07974855289928


ValueError: DFT energy of localized components not matching supersystem DFT.

In [ ]:
"""Function to perform subsystem RKS DFT calculation."""
logger.debug("Calculating active and environment subsystem terms.")

def _rks_components(
    rks_system: Localizer, subsystem_dm: np.ndarray,
) -> Tuple[float, float, np.ndarray, np.ndarray, np.ndarray]:
    """Calculate the components of subsystem energy from a RKS DFT calculation.
    For a given density matrix this function returns the electronic energy, exchange correlation energy and
    J,K, V_xc matrices.
    Args:
        dm_matrix (np.ndarray): density matrix (to calculate all matrices from)
    Returns:
        Energy_elec (float): DFT energy defubed by input density matrix
        e_xc (float): exchange correlation energy defined by input density matrix
        J_mat (np.ndarray): J_matrix defined by input density matrix
    """
    dm_matrix = subsystem_dm
    # It seems that PySCF lumps J and K in the J array
    two_e_term = rks_system.get_veff(dm=dm_matrix)
    j_mat = two_e_term.vj
    # k_mat = np.zeros_like(j_mat)

    e_xc = two_e_term.exc
    # v_xc = two_e_term - j_mat

    energy_elec = (
        np.einsum("ij,ji->", rks_system.get_hcore(), dm_matrix)
        + two_e_term.ecoul
        + two_e_term.exc
    )

    # if check_E_with_pyscf:
    #     energy_elec_pyscf = driver._global_ks.energy_elec(dm=dm_matrix)[0]
    #     if not np.isclose(energy_elec_pyscf, energy_elec):
    #         raise ValueError("Energy calculation incorrect")

    return energy_elec, e_xc, j_mat

(driver.e_act, e_xc_act, j_act) = _rks_components(
    driver._global_ks, driver.localized_system.dm_active
)
(driver.e_env, e_xc_env, j_env) = _rks_components(
    driver._global_ks, driver.localized_system.dm_enviro
)
# Computing cross subsystem terms
logger.debug("Calculating two electron cross subsystem energy.")

two_e_term_total = driver._global_ks.get_veff(
    dm=driver.localized_system.dm_active + driver.localized_system.dm_enviro
)
e_xc_total = two_e_term_total.exc

j_cross = 0.5 * (
    np.einsum("ij,ij", driver.localized_system.dm_active, j_env)
    + np.einsum("ij,ij", driver.localized_system.dm_enviro, j_act)
)
# Because of projection
k_cross = 0.0

xc_cross = e_xc_total - e_xc_act - e_xc_env

# overall two_electron cross energy
driver.two_e_cross = j_cross + k_cross + xc_cross
print(f"{e_xc_total=}, {alpha_e_xc_act=}, {alpha_e_xc_env=}, {j_cross=}, {k_cross=}, {xc_cross=}")

energy_DFT_components = (
    driver.e_act + driver.e_env + driver.two_e_cross + driver._global_ks.energy_nuc()
)
logger.info("RKS components")
logger.info(driver.e_act)
logger.info(driver.e_env)
logger.info(driver.two_e_cross)
logger.info(driver._global_ks.energy_nuc())
print(energy_DFT_components, driver._global_ks.e_tot)
if not np.isclose(energy_DFT_components, driver._global_ks.e_tot):

    raise ValueError(
        "DFT energy of localized components not matching supersystem DFT"
    )

In [ ]:

logger.debug("Getting global DFT potential to optimize embedded calc in.")
g_act_and_env = driver._global_ks.get_veff(
    dm=(localized_system.dm_active + localized_system.dm_enviro)
)
g_act = driver._global_ks.get_veff(dm=localized_system.dm_active)
driver._dft_potential = g_act_and_env - g_act
logger.info(f"DFT potential average {np.mean(driver._dft_potential)}.")

# To add a projector, put it in this dict with a function
# if we want any more it's also time to turn them into a class
embeddings: Dict[str, callable] = {
    "huzinaga": driver._huzinaga_embed,
    "mu": driver._mu_embed,
}
if driver.projector in embeddings:
    embeddings = {driver.projector: embeddings[driver.projector]}

# This is reverse so that huz can be initialised with mu
for name in sorted(embeddings, reverse=True):
    logger.debug(f"Runnning embedding with {name} projector.")
    setattr(driver, "_" + name, {})
    result = getattr(driver, "_" + name)

    embedding_method = embeddings[name]
    local_rhf = driver._init_local_rhf()

    if init_huzinaga_rhf_with_mu and (name == "huzinaga"):
        logger.debug("Initializing huzinaga with mu-shift.")
        # seed huzinaga calc with mu result!
        result["v_emb"], result["scf"] = embedding_method(
            local_rhf, dmat_initial_guess=driver._mu["scf"].make_rdm1()
        )
    else:
        result["v_emb"], result["scf"] = embedding_method(local_rhf)

    result["mo_energies_emb_pre_del"] = local_rhf.mo_energy
    result["scf"] = driver._delete_environment(result["scf"], name)
    result["mo_energies_emb_post_del"] = local_rhf.mo_energy

    logger.info(f"V emb mean {name}: {np.mean(result['v_emb'])}")

    # calculate correction
    result["correction"] = np.einsum(
        "ij,ij", result["v_emb"], localized_system.dm_active
    )
    result["e_rhf"] = (
        result["scf"].e_tot
        + driver.e_env
        + driver.two_e_cross
        - result["correction"]
    )
    logger.info(f"RHF energy: {result['e_rhf']}")

    # classical energy
    result["classical_energy"] = (
        driver.e_env + driver.two_e_cross + e_nuc - result["correction"]
    )

    # Calculate ccsd or fci energy
    if driver.run_ccsd_emb is True:
        logger.debug("Performing CCSD-in-DFT embedding.")
        ccsd_emb, e_ccsd_corr = driver._run_emb_CCSD(
            result["scf"], frozen_orb_list=None
        )
        result["e_ccsd"] = (
            ccsd_emb.e_hf
            + e_ccsd_corr
            + driver.e_env
            + driver.two_e_cross
            - result["correction"]
        )
        logger.info(f"CCSD Energy {name}:\t{result['e_ccsd']}")

    if driver.run_fci_emb is True:
        logger.debug("Performing FCI-in-DFT embedding.")
        fci_emb = driver._run_emb_FCI(result["scf"], frozen_orb_list=None)
        result["e_fci"] = (
            (fci_emb.e_tot)
            + driver.e_env
            + driver.two_e_cross
            - result["correction"]
        )
        logger.info(f"FCI Energy {name}:\t{result['e_fci']}")

    if driver.run_dft_in_dft is True:
        did = driver.embed_dft_in_dft(driver._global_ks.xc, embedding_method)
        result["e_dft_in_dft"] = did["e_rks"]

if driver.projector == "both":
    logger.warning(
        "Outputting both mu and huzinaga embedding results as tuple."
    )
    driver.embedded_scf = (
        driver._mu["scf"],
        driver._huzinaga["scf"],
    )
    driver.classical_energy = (
        driver._mu["classical_energy"],
        driver._huzinaga["classical_energy"],
    )
elif driver.projector == "mu":
    driver.embedded_scf = driver._mu["scf"]
    driver.classical_energy = driver._mu["classical_energy"]
elif driver.projector == "huzinaga":
    driver.embedded_scf = driver._huzinaga["scf"]
    driver.classical_energy = driver._huzinaga["classical_energy"]

logger.info("Embedding complete.")